In [1]:
%tensorflow_version 1.x 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
import pickle
import os
import shutil
import pandas as pd
import numpy as np
import cv2
import tqdm
%tensorflow_version 1.x 
from keras.utils import np_utils

TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
import keras
import tensorflow

In [3]:
tensorflow.__version__

'1.15.2'

In [4]:
keras.__version__


'2.2.5'

# Setup

In [0]:

dataset_path = "path/to/zipped/data/NLP.zip"


Unpacking the data into data folder


In [0]:
shutil.unpack_archive(dataset_path, "data/")

Download the pickled glove vector from kaggle and add the link here

In [0]:
def loadData(filename):
    file_ptr = open(filename, 'rb')
    loaded_obj = pickle.load(file_ptr)
    return loaded_obj

embeddings_index  = loadData("/path/to/glove.840B.300d.pkl")


In [0]:
dataset_path = "data/"

read the dataset into memory

In [0]:
train_df = pd.read_csv(dataset_path + "train.csv")
test_df = pd.read_csv(dataset_path + "test.csv")

In [111]:
len(pd.read_csv(dataset_path + "train.csv"))

5279

In [110]:
len(pd.read_csv(dataset_path + "test.csv"))

2924

In [12]:
print(train_df.columns)

Index(['unique_hash', 'text', 'drug', 'sentiment'], dtype='object')


In [0]:
train_df, val_df = train_test_split(train_df, test_size=0.1)

A function to convert text to glove vectors

In [19]:
# Convert values to embeddings
def text_to_array(text):
    empyt_emb = np.zeros(300)
    text = text[:-1].split()[:30]
    embeds = [embeddings_index.get(x, empyt_emb) for x in text]
    embeds+= [empyt_emb] * (30 - len(embeds))
    return np.array(embeds)

# train_vects = [text_to_array(X_text) for X_text in tqdm(train_df["question_text"])]
val_vects = np.array([text_to_array(X_text) for X_text in tqdm.tqdm(val_df["text"][:3000])])
val_vects2 = np.array([text_to_array(X_text) for X_text in tqdm.tqdm(val_df["drug"][:3000])])
val_y = np.array(val_df["sentiment"][:3000])


100%|██████████| 528/528 [00:00<00:00, 35688.73it/s]


In [109]:
print(len(train_df))

4751


convert to one hot

In [49]:
print (val_y.shape)
val_y = np_utils.to_categorical(val_y, 3)
print (val_y.shape)

(528,)
(528, 3)


dataset generator

In [0]:
batch_size = 128

def batch_gen(train_df):
    n_batches = math.ceil(len(train_df) / batch_size)
    while True: 
        train_df = train_df.sample(frac=1.)  # Shuffle the data.
        for i in range(n_batches):
            texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
            texts2 = train_df.iloc[i*batch_size:(i+1)*batch_size, 2]
            text_arr = np.array([text_to_array(text) for text in texts])
            text_arr2 = np.array([text_to_array(text) for text in texts2])
            
            yield [text_arr,text_arr2], np_utils.to_categorical(np.array(train_df["sentiment"][i*batch_size:(i+1)*batch_size]),3)


# Training

The model has two inputs one for the sentence and the other for the name of the entity

In [0]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional


In [0]:

from keras.layers import concatenate
from keras.layers import Input, Dense
from keras.models import Model


def mo():
  inputs = Input(shape=(30, 300))
  inputs_ent = Input(shape=(30, 300))
  # x = Dense(8, activation="relu")
  x = Bidirectional(LSTM(64, return_sequences=True))(inputs)
  y = Bidirectional(LSTM(64, return_sequences=True))(inputs_ent)
  x = Bidirectional(LSTM(64, return_sequences=True))(x)
  y = Bidirectional(LSTM(64, return_sequences=True))(y)
  x = Bidirectional(LSTM(64))(x)
  y = Bidirectional(LSTM(64))(y)
  x =Dense(24, activation='relu')(x)
  y =Dense(24 , activation='relu')(y)  
  x =Dense(12, activation='relu')(x)
  y =Dense(12 , activation='relu')(y)
  z = concatenate([x,y], axis = -1)
  z =Dense(3 , activation='sigmoid')(z)
  model = Model([inputs,inputs_ent], z)
  return model

In [0]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model = mo()

In [99]:
print(model.summary())

Model: "model_12"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_41 (InputLayer)           (None, 30, 300)      0                                            
__________________________________________________________________________________________________
input_42 (InputLayer)           (None, 30, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_78 (Bidirectional (None, 30, 128)      186880      input_41[0][0]                   
__________________________________________________________________________________________________
bidirectional_79 (Bidirectional (None, 30, 128)      186880      input_42[0][0]                   
___________________________________________________________________________________________

In [0]:

model.compile(loss='binary_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [101]:
from keras.callbacks import ReduceLROnPlateau

lr_reduce = ReduceLROnPlateau(monitor='val_acc', factor=0.1, epsilon=0.0001, patience=3, verbose=1)

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1335: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


In [102]:
mg = batch_gen(train_df)
model.fit_generator(mg, epochs=10,
                    steps_per_epoch=math.ceil(len(train_df) / batch_size),
                    validation_data=([val_vects,val_vects2], val_y),
                    callbacks=[lr_reduce],

                    verbose=True)

Epoch 1/10
38/38 [==============================] - 43s 1s/step - loss: 0.6413 - acc: 0.7646 - val_loss: 0.5726 - val_acc: 0.8093
Epoch 2/10
38/38 [==============================] - 25s 668ms/step - loss: 0.5151 - acc: 0.8193 - val_loss: 0.4852 - val_acc: 0.8093
Epoch 3/10
38/38 [==============================] - 25s 656ms/step - loss: 0.4662 - acc: 0.8173 - val_loss: 0.4710 - val_acc: 0.8093
Epoch 4/10
38/38 [==============================] - 25s 659ms/step - loss: 0.4655 - acc: 0.8143 - val_loss: 0.4704 - val_acc: 0.8093

Epoch 00004: ReduceLROnPlateau reducing learning rate to 0.0009999999776482583.
Epoch 5/10
38/38 [==============================] - 25s 662ms/step - loss: 0.4613 - acc: 0.8173 - val_loss: 0.4700 - val_acc: 0.8093
Epoch 6/10
38/38 [==============================] - 25s 648ms/step - loss: 0.4587 - acc: 0.8193 - val_loss: 0.4699 - val_acc: 0.8093
Epoch 7/10
38/38 [==============================] - 25s 653ms/step - loss: 0.4625 - acc: 0.8163 - val_loss: 0.4700 - val_acc

In [0]:
model.save('my_model.h5')

# Inference

In [103]:
test_vects = np.array([text_to_array(X_text) for X_text in tqdm.tqdm(test_df["text"][:3000])])
test_vects2 = np.array([text_to_array(X_text) for X_text in tqdm.tqdm(test_df["drug"][:3000])])
# val_y = np.array(val_df["sentiment"][:3000])

100%|██████████| 2924/2924 [00:00<00:00, 32368.87it/s]


In [107]:
print(test_vects.shape, test_vects2.shape)

(2924, 30, 300) (2924, 30, 300)


In [120]:
test_labels = []
for i in model.predict([test_vects, test_vects2]):
  print(i)
  test_labels.append(np.argmax( i))

[0.12345067 0.16883901 0.69802785]
[0.11703658 0.16214925 0.7176714 ]
[0.1165576  0.16510382 0.72032535]
[0.11738774 0.16149184 0.72073686]
[0.11675802 0.16343674 0.7173153 ]
[0.12458849 0.16053155 0.7097107 ]
[0.11677083 0.16597378 0.71712077]
[0.11862499 0.16079307 0.72217786]
[0.11610845 0.16515538 0.71615446]
[0.11564752 0.16488573 0.7168041 ]
[0.11852005 0.16229081 0.71904767]
[0.11916688 0.1600776  0.720063  ]
[0.11687517 0.16107228 0.7203885 ]
[0.11707723 0.1667608  0.7136064 ]
[0.11830246 0.16830805 0.7097739 ]
[0.11653608 0.16207129 0.7185925 ]
[0.1220504  0.16113898 0.7109762 ]
[0.11517456 0.16850275 0.71899617]
[0.116413   0.15976173 0.72473294]
[0.11935714 0.16372994 0.71344477]
[0.11697677 0.16632673 0.7128049 ]
[0.11741009 0.16550949 0.7153177 ]
[0.12291238 0.15634644 0.7207186 ]
[0.11952871 0.15900955 0.7207631 ]
[0.11825162 0.16572484 0.7099904 ]
[0.1215843  0.16098398 0.71284837]
[0.12059575 0.16272953 0.71443   ]
[0.11944231 0.16714177 0.70824647]
[0.11850476 0.163181

In [115]:
test_labels

[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,


In [116]:
len(test_labels)

2924

In [117]:
len(test_df)

2924

In [0]:
test_df['sentiment'] = test_labels

In [119]:
print(test_df)

                                   unique_hash  ... sentiment
0     9e9a8166b84114aca147bf409f6f956635034c08  ...         2
1     e747e6822c867571afe7b907b51f0f2ca67b0e1a  ...         2
2     50b6d851bcff4f35afe354937949e9948975adf7  ...         2
3     7f82ec2176ae6ab0b5d20b5ffc767ac829f384ae  ...         2
4     8b37d169dee5bdae27060949242fb54feb6a7f7f  ...         2
...                                        ...  ...       ...
2919  ac6e60bec9162ae66effd29a0dc9ad11ff966df6  ...         2
2920  12afabb6210825308ead9894916abdfc912d7c43  ...         2
2921  021bb88c92a71229288304d691d53c3ff7004e4b  ...         2
2922  9936efcb83eded79fe9454df188edd7b96c6109e  ...         2
2923  05402df12d6769d7f38ab40e0b81464e65e1df0a  ...         2

[2924 rows x 4 columns]


In [0]:
model.save('/gdrive/My Drive/Caw/modelnlp')

In [123]:
!ls gdrive/My\ Drive/Caw/model

ls: cannot access 'drive/My Drive/Caw/model': No such file or directory


In [0]:
test_df.to_csv('/gdrive/My Drive/Caw/submission2.csv', index=False)